# IRF2BP2 CHIPS

In [ ]:
import os
from IPython.display import IFrame
import pandas as pd
import sys
sys.path.append('../..')
from JKBio.epigenetics import ChIP_helper as chip
from JKBio import Helper as helper
import igv
import numpy as np
import pyBigWig
import itertools
from sklearn.cluster import AgglomerativeClustering
from sklearn.mixture import GaussianMixture
from bokeh.plotting import *
from sklearn.manifold import MDS, TSNE
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
output_notebook()
%load_ext autoreload
%autoreload 2

In [ ]:
bfolder = '../../data/recalib_bigwig_rep1/'
folder = '../../data/diffBinding_rep1/diffPeaks/'

In [ ]:
# Can we find a correlation between each MED1 replicate

In [ ]:
ls ../../data/diffBinding_rep1/diffPeaks/MV411_IRF2BP2_DMSO_MED1*

In [ ]:
ls ../../data/diffBinding_rep2/diffPeaks/mp796*

In [ ]:
ls ../../data/IRF2BP2_degraded_3/diffPeaks/mp835*

In [ ]:
diff1 = pd.DataFrame()
for val in os.popen('ls ../../data/diffBinding_rep1/diffPeaks/MV411_IRF2BP2_DMSO_MED1*').read().split('\n')[:-1]:
    a = pd.read_csv(val, sep='\t', header=0, names=['chr','start','end','name','val']).sort_values(by=['val'])
    diff1 = diff1.append(a).reset_index(drop=True)

In [ ]:
diff2 = pd.DataFrame()
for val in os.popen('ls ../../data/diffBinding_rep2/diffPeaks/mp796*').read().split('\n')[:-1]:
    a = pd.read_csv(val, sep='\t', header=0, names=['chr','start','end','name','val']).sort_values(by=['val'])
    diff2 = diff2.append(a).reset_index(drop=True)

In [ ]:
diff3 = pd.DataFrame()
for val in os.popen('ls ../../data/IRF2BP2_degraded_3/diffPeaks/mp835*').read().split('\n')[:-1]:
    a = pd.read_csv(val, sep='\t', header=0, names=['chr','start','end','name','val']).sort_values(by=['val'])
    diff3 = diff3.append(a).reset_index(drop=True)

In [ ]:
diff3['type'] = [val.split('_')[-2] for val in diff3.name.tolist()]
diff2['type'] = [val.split('_')[-2] for val in diff2.name.tolist()]
diff1['type'] = [val.split('_')[-2] for val in diff1.name.tolist()]

In [ ]:
diff1

In [ ]:
len(pd.read_csv('../../data/diffBinding_rep1/diffPeaks/MV411_IRF2BP2_DMSO_MED1_R1_vs_MV411_IRF2BP2_VHL_MED1_R1_c3.0_cond2.bed', sep='\t', header=0, names=['chr','start','end','name','val']).sort_values(by=['val']))

In [ ]:
diff2

In [ ]:
len(pd.read_csv('../../data/diffBinding_rep2/diffPeaks/mp796_R1_vs_mp804_R1_c3.0_cond1.bed', sep='\t', header=0, names=['chr','start','end','name','val']).sort_values(by=['val']))

In [ ]:
len(pd.read_csv('../../data/diffBinding_rep2/diffPeaks/mp796_R1_vs_mp804_R1_c3.0_common.bed', sep='\t', header=0, names=['chr','start','end','name','val']).sort_values(by=['val']))

In [ ]:
diff3

In [ ]:
from pybedtools import BedTool
cm= "../../data/diffBinding_rep1/diffPeaks/MV411_IRF2BP2_DMSO_MED1_R1_vs_MV411_IRF2BP2_VHL_MED1_R1_c3.0_common.bed"
cd1="../../data/diffBinding_rep1/diffPeaks/MV411_IRF2BP2_DMSO_MED1_R1_vs_MV411_IRF2BP2_VHL_MED1_R1_c3.0_cond1.bed"
cd2="../../data/diffBinding_rep1/diffPeaks/MV411_IRF2BP2_DMSO_MED1_R1_vs_MV411_IRF2BP2_VHL_MED1_R1_c3.0_cond2.bed"
merged1 = BedTool(cm).cat(BedTool(cd1), postmerge=False).cat(BedTool(cd2), postmerge=False)

cm="../../data/IRF2BP2_degraded_3/diffPeaks/mp835-MV411_IRF2BP_DMSO_6h-MED1-r3_R1_vs_mp842-MV411_IRF2BP_VHL_6h-MED1-r3_R1_c3.0_common.bed"
cd1="../../data/IRF2BP2_degraded_3/diffPeaks/mp835-MV411_IRF2BP_DMSO_6h-MED1-r3_R1_vs_mp842-MV411_IRF2BP_VHL_6h-MED1-r3_R1_c3.0_cond1.bed"
cd2="../../data/IRF2BP2_degraded_3/diffPeaks/mp835-MV411_IRF2BP_DMSO_6h-MED1-r3_R1_vs_mp842-MV411_IRF2BP_VHL_6h-MED1-r3_R1_c3.0_cond2.bed"
merged3 = BedTool(cm).cat(BedTool(cd1), postmerge=False).cat(BedTool(cd2), postmerge=False)

In [ ]:
len(merged1)

In [ ]:
len(merged3)

In [ ]:
df = merged3.intersect(merged1,wa=True,wb=True).to_dataframe(names=['chrA','startA','endA','nameA','valA','chrB','startB','endB','nameB','valB'])
df

In [ ]:
df['typeA'] = [i.split('_')[-2] for i in df['nameA']]

In [ ]:
df['typeB'] = [i.split('_')[-2] for i in df['nameB']]

In [ ]:
df

In [ ]:
df[df.typeA=='common'][df.typeB=='common']

In [ ]:
i=0
p_valA=0
p_valB=0
p_valC=0
p_valD=0
p_valE=0
sA=0
sB=0
sC=0
sD=0
sE=0
for k, val in df.iterrows():
    if (val.typeA == 'cond2' and val.typeB=="cond1") or (val.typeA=="cond1" and val.typeB=="cond2"):
        i+=1
    if val.typeA == 'common' and val.typeB in ["cond1","cond2"]:
        p_valA += val.valA 
        sA+=1
        p_valD += val.valB
        sD+=1
    if (val.typeA == 'cond2' and val.typeB=="cond2") or (val.typeA=="cond1" and val.typeB=="cond1"):
        p_valE += val.valA
        sE+=1
    if val.typeB == 'common' and val.typeA in ["cond1","cond2"]:
        p_valB += val.valB 
        sB+=1
totpvalA = df[df.typeA=='common'][df.typeB=='common'].valA.mean()
totpvalB = df[df.typeA=='common'][df.typeB=='common'].valB.mean()

We can see that we don't have much opposing results (1%),

We also have that there is a decrease of 10% of how likely an enhancer bound by MED1 is to be common between the two condition in replicate3 if it was not a common gene in replicate1 compared to the global average. It is 25% lower when they are considered common in both replicate.

It tends to show that there is a difference at the level of the experiment but we still see some correlation (maybe do some plot)
It is mainly due to the spike In control which had much less variable ratio in rep3. It is a clear bias of experiment 3 to have very low difference between its spikeIns. We can see that someone tried to compensate for the high variance in MED1 rep2. It was not a good idea as it bias the entire experiment

In [ ]:
import numpy as np

In [ ]:
p_valD/sD

In [ ]:
p_valC/sC

In [ ]:
p_valB/sB

In [ ]:
print(' We have a percentage of opposing results of: '+ str(i/len(df))+'\n\n We have a decrease of '+ str((1-((p_valA/sA)/totpvalA))*100) +' of how likely an enhancer bound by MED1 is to be common between the two condition in replicate3 if it was not a common gene in replicate1 compared to the global average, for a total number of ' +str(sA)+' and a variance of '+str(np.var(p_valA))+'.\n\n Similarly,  It is '+ str((1-((p_valB/sB)/totpvalB))*100) +' lower when they are considered common in both replicate (with'+str(sB)+' replicates). \n\n This value is of '+str(p_valC/sC)+' when considering all common replicates with total number of '+str(sC)+' datapoints.\n\n Finally There is an increase in accuracy of '+str((((p_valD/sD)/(p_valE/sE))-1)*100)+' in confidence of overbound peaks in rep1 when these peaks are not overbound in rep3 vs when they are in both')

let's try to make a plot (how things correlate)

In [ ]:
import seaborn as sns
sns.lmplot(data=df[df.typeA=='cond1'][df.typeB=='cond1'],x='valA',y='valB')
sns.lmplot(data=df[df.typeA=='common'][df.typeB=='cond1'],x='valA',y='valB')
sns.lmplot(data=df[df.typeA=='common'][df.typeB=='common'],x='valA',y='valB')
sns.lmplot(data=df[df.typeA=='cond2'][df.typeB=='common'],x='valA',y='valB')

As expected, plots are definitely unconclusives..

# Making profiles plot

In [ ]:
! ls -alh $folder

In [ ]:
peaks = ! ls $folder

In [ ]:
bigwig = ! ls $bfolder
num = 8

In [ ]:
bigwig

In [ ]:
bigwig=[(bfolder+bigwig[i],bfolder+bigwig[i+num]) for i in range(num)]

In [ ]:
bigwig

In [ ]:
peaks = [p for p in helper.grouped([folder+p for p in peaks],3)]

In [ ]:
! mkdir temp

In [ ]:
cov = {}
fig = {}
for i,peak in enumerate(peaks):
    p = pd.read_csv(peak[1],sep='\t',skiprows=1,header=None,
            names=['chrom','start','end','name','foldchange']).sort_values(by='foldchange',ascending=False).iloc[:500]
    p2 =pd.read_csv(peak[0],sep='\t',skiprows=1,header=None,
            names=['chrom','start','end','name','foldchange']).iloc[:500]
    p = p.append(p2)
    p2 =pd.read_csv(peak[2],sep='\t',skiprows=1,header=None,
            names=['chrom','start','end','name','foldchange']).sort_values(by='foldchange',ascending=True).iloc[-500:]
    p = p.append(p2)
    
    cov.update(chip.computePeaksAt(p,bigwig[i], window=1000, numpeaks=1500, numthreads=8,
                   width=5, length=10, name='temp/peaksat'+str(i)+'.png', sort=False)[0])

In [ ]:
for k,i in cov.items():
    print(k, i.max())

In [ ]:
pd.read_csv(peak[1],sep='\t')